In [1]:
import sys 
import time
from geopy.geocoders import Nominatim, GoogleV3, Bing
import folium
from bs4 import BeautifulSoup
import requests
import numpy as np
import googleCreds
import pandas as pd
geolocator = GoogleV3(api_key = googleCreds.GOOGLE_API_KEY)

### 1. Companies' markers as a feature group

In [2]:
dfCompanies = pd.read_csv("dfCompanies.csv")
dfCompanies.dtypes

name        object
lt         float64
lg         float64
address     object
dtype: object

In [3]:
#Creating a dictionary for a future dataset (dfCompanies) and adding markers to the map
feature_group1 = folium.FeatureGroup(name='Companies')

#I need this location to center the future map 
location = geolocator.geocode("Milano Italy")

# map1 = folium.Map(
#     location=[location.latitude, location.longitude],
#     tiles='cartodbpositron',
#     zoom_start=13)
tooltip = 'Click me!'

for index, loc in dfCompanies.iterrows():
#     print([loc["lt"], loc["lg"]])
#     print(f'<i>{loc.name}+": "+{loc.address}</i>')
#     input()
    if "Milano" in set(loc.address.split(" ")) or "MI," in set(loc.address.split(" ")):
        #Adding marker
        folium.Marker([loc["lt"], loc["lg"]], 
                      popup=f'<i>{loc.name}+": "+{loc.address}</i>', 
                      tooltip=tooltip).add_to(feature_group1)
feature_groups = []
feature_groups.append(feature_group1)

### 2. Median and mean location markers as a feature group

In [4]:
meanLat = dfCompanies["lt"].mean()
meanLon = dfCompanies["lg"].mean()

medianLat = dfCompanies["lt"].median()
medianLon = dfCompanies["lg"].median()

medianLocation = (medianLat, medianLon)
meanLocation = (meanLat, meanLon)

In [5]:
feature_group2 = folium.FeatureGroup(name="Optimal Locations based on companies")
folium.Marker([meanLat, meanLon], 
                      popup='<i>Mean location</i>',
                      radius= 50,
                      tooltip=tooltip, 
                      icon=folium.Icon(color='green', 
                                       icon='glyphicon-ok-circle')).add_to(feature_group2)

folium.Marker([medianLat, medianLon], 
                      popup='<i>Median location</i>',
                      radius= 50,
                      tooltip=tooltip, 
                      icon=folium.Icon(color='green', 
                                       icon='glyphicon-ok-circle')).add_to(feature_group2)

#feature_group2.add_to(map1)
feature_groups.append(feature_group2)

### 3. Zones of Milan

In [6]:
milanodistricts = "zonedecentramento.geojson" 
#Source: https://dati.comune.milano.it/dataset/ds153-infogeo-zone-localizzazione-2013
#Last Access Date: 11/08/2020

feature_group3 = folium.FeatureGroup(name='Districts (Zones)')
folium.GeoJson(
    milanodistricts,
    name='geojson').add_to(feature_group3)


feature_groups.append(feature_group3)
# folium.TopoJson(
#     milanodistricts,
#     'objects.antarctic_ice_shelf',
#     name='topojson').add_to(map1)
# folium.LayerControl().add_to(map1)

# map1.choropleth(
#   geo_data=milanodistricts,
#   fill_color='YlOrRd', 
#   fill_opacity=0.7, 
#   line_opacity=0.2
# )
# feature_group3.add_to(map1)
# folium.LayerControl(collapsed = False).add_to(map1)
# map1

### 4. Green Zones

I had to use this map(layer) as a foundation because I couldn't find out how to make choropleth layer toggleable.

In [7]:
dfGreen = pd.read_csv("dfGreen.csv")
dfGreen

,ZONADEC,GreenArea,TotalArea,GreenConc
0,1,1.108470,9.67,0.114630
1,2,1.067198,12.58,0.084833
2,3,1.732050,14.23,0.121718
3,4,2.133899,20.95,0.101857
4,5,1.745979,29.87,0.058453
5,6,2.125662,18.28,0.116283
6,7,5.294581,31.34,0.168940
7,8,3.549903,23.72,0.149659
8,9,3.677692,21.12,0.174133


In [8]:
mapGreen = folium.Map(
    location=[location.latitude, location.longitude],
    zoom_start=12,
    tiles='cartodbpositron')


choropleth = folium.Choropleth('zonedecentramento.geojson',
                  name='Vegetation concentration',
                  data=dfGreen,
                  key_on="properties.ZONADEC",
                  columns=['ZONADEC', 'GreenConc'],
                  fill_color='YlGn',
                  fill_opacity=0.7,
                  line_opacity=0.2,
                  legend_name='Vegetation Concentration (green area per total area)',
                  show = False
).add_to(mapGreen)

choropleth.geojson.add_child(
    folium.features.GeoJsonTooltip(['ZONADEC'])
)
# feature_group1.add_to(mapGreen)
# feature_group2.add_to(mapGreen)
# feature_group3.add_to(mapGreen)

### 5. Environmental monitoring stations 

In [9]:
dfAir3=pd.read_csv("5_dfAirScaledMedianStations.csv")
dfAir3

,id_amat,valore,normv,name,coords,lt,lg
0,2,12.0,0.150000,via Pascal *,"[45.4740982055664, 9.23478031158447]",45.474098,9.23478
1,3,2.2,0.281250,viale Liguria,"[45.4441986083984, 9.16944026947021]",45.444199,9.16944
2,4,8.5,0.218750,viale Marche,"[45.4962997436523, 9.19083976745605]",45.496300,9.19084
3,6,14.0,0.200000,via Senato *,"[45.4705009460449, 9.19791984558105]",45.470501,9.19792
4,7,52.0,0.261397,Verziere,"[45.4635009765625, 9.19534015655518]",45.463501,9.19534


In [10]:
import json
with open('qaria_stazione.geojson') as data_file:
    data = json.load(data_file)

def color_producer(value, mx, inv = False):
    if inv == False:
        if value < 0.3*mx:
            return 'green'
        elif value < 0.75*mx:
            return 'orange'
        else:
            return 'red'
    else:
        if value < 0.3*mx:
            return 'red'
        elif value < 0.75*mx:
            return 'orange'
        else:
            return 'green'
        
feature_group5 = folium.FeatureGroup(name='Environmental monitoring stations ')
# mapB = folium.Map(
#     location=[location.latitude, location.longitude],
#     zoom_start=13,
#     tiles='cartodbpositron')
mx = max(dfAir3["normv"])

    
for i, row in dfAir3.iterrows(): #data["features"]):
    #print(f["geometry"]["coordinates"])
    try:
        folium.Circle(location = [data["features"][int(row["id_amat"])-1]["geometry"]["coordinates"][1],
                                  data["features"][int(row["id_amat"])-1]["geometry"]["coordinates"][0]], 
                      radius = 1500,
                      fill = True,
                      fill_color = color_producer(row["normv"], mx),
                      weight = 0,
                      popup = f"Station #{i}. Conc: "+str(round(row["normv"],2))).add_to(feature_group5)
    except:
        folium.Circle(location = [data["features"][int(row["id_amat"])-1]["geometry"]["coordinates"][1],
                                  data["features"][int(row["id_amat"])-1]["geometry"]["coordinates"][0]], 
                      radius = 1500,
                      fill = True,
                      fill_color = "grey",
                      weight = 0, 
                      popup = "NA").add_to(feature_group5)

# feature_group5.add_to(mapGreen)
feature_groups.append(feature_group5)

### 6. Predicted Air Quality scores

In [11]:
minmaxlat = [45.39, 45.53]
minmaxlong = [9.08, 9.29]

In [12]:
from folium.plugins import HeatMap


dct = {"lt":[],
      "lg": []}
for i in np.arange(minmaxlat[0], minmaxlat[1], 0.01):
    for j in np.arange(minmaxlong[0], minmaxlong[1], 0.01):
        dct["lt"].append(i)
        dct["lg"].append(j)
        
dfHeatmap = pd.DataFrame(dct)
dfHeatmap = dfHeatmap.append(dfAir3[["lt","lg"]])

In [13]:
import pickle
pkl_filename = "lr.pkl"
with open(pkl_filename, 'rb') as file:
    lr = pickle.load(file)

In [14]:
from sklearn.preprocessing import PolynomialFeatures

pf = PolynomialFeatures(degree=3)

In [15]:
dfHeatmap["contamination"] = lr.predict(pf.fit_transform(dfHeatmap))
dfHeatmap

,lt,lg,contamination
0,45.390000,9.08000,-1.191735
1,45.390000,9.09000,-0.801161
2,45.390000,9.10000,-0.420527
3,45.390000,9.11000,-0.049804
4,45.390000,9.12000,0.311036
...,...,...,...
0,45.474098,9.23478,0.150000
1,45.444199,9.16944,0.281250
2,45.496300,9.19084,0.218750
3,45.470501,9.19792,0.200000


In [16]:
dfHeatmap["contamination"] = dfHeatmap["contamination"].map(lambda x: max(0, x))
dfHeatmap

,lt,lg,contamination
0,45.390000,9.08000,0.000000
1,45.390000,9.09000,0.000000
2,45.390000,9.10000,0.000000
3,45.390000,9.11000,0.000000
4,45.390000,9.12000,0.311036
...,...,...,...
0,45.474098,9.23478,0.150000
1,45.444199,9.16944,0.281250
2,45.496300,9.19084,0.218750
3,45.470501,9.19792,0.200000


In [17]:
import branca.colormap as cm
from collections import defaultdict
steps = 20
#color_map=cm.linear.Blues_03.scale(0,1).to_step(steps)
color_map=cm.linear.YlOrRd_09.scale(0,1).to_step(steps)

feature_group6 = folium.FeatureGroup(name='Air Contamination Heatmap')
gradient_map=defaultdict(dict)
for i in range(steps):
    gradient_map[1/steps*i] = color_map.rgb_hex_str(1/steps*i)

# mapB = folium.Map(
#     location=[location.latitude, location.longitude],
#     zoom_start=13,
#     tiles='cartodbpositron')

HeatMap(dfHeatmap, min_opacity = 0.3, 
        radius =50,
        blur = 50,
       gradient = gradient_map).add_to(folium.FeatureGroup(name='Heat Map').add_to(feature_group6))

#feature_group6.add_to(mapGreen)
feature_groups.append(feature_group6)

### 7. Apartments for rent (scored)

In [18]:
minPrice = 1000
maxPrice = 2000

In [19]:
dfObjects=pd.read_csv("dfObjects.csv")
dfObjects = dfObjects[(dfObjects["price"]>=minPrice) & (dfObjects["price"]<=maxPrice)]
dfObjects = dfObjects[dfObjects["type"] == "Bilocale"]
dfObjects.head()

,id,type,address,price,coords,contamination,distanceToDangerZone,distanceToMedianJobLocation,Zone,GreenConc,lt,lg,priceLogScaled,totalScore
2,link_ad_79920811,Bilocale,"corso Indipendenza 18, Indipendenza, Milano",1350,"[45.467451, 9.214442499999999]",0.269303,3.128875,1.924670,4.0,0.101857,45.467451,9.214443,0.331556,3.526012
5,link_ad_83000949,Bilocale,"via Carlo Imbonati 64, Dergano, Milano",1300,"[45.50395839999999, 9.182326399999999]",0.367617,1.781589,3.849130,9.0,0.174133,45.503958,9.182326,0.318958,-12.899910
12,link_ad_83770907,Bilocale,"viale Legioni Romane 8, Bande Nere, Milano",1050,"[45.46091029999999, 9.1343716]",-0.230637,2.079047,4.447070,6.0,0.116283,45.460910,9.134372,0.247665,-11.343839
13,link_ad_82869489,Bilocale,"corso Magenta, San Vittore, Milano",1500,"[45.4657096, 9.1717594]",0.113508,1.818443,1.490735,1.0,0.114630,45.465710,9.171759,0.366726,-0.189478
14,link_ad_81424435,Bilocale,"via Pinamonte da Vimercate, Moscova, Milano",1250,"[45.4782881, 9.1823255]",0.143367,3.243976,1.120469,1.0,0.114630,45.478288,9.182326,0.305865,8.944522


In [20]:
dfObjects.dtypes

id                              object
type                            object
address                         object
price                            int64
coords                          object
contamination                  float64
distanceToDangerZone           float64
distanceToMedianJobLocation    float64
Zone                           float64
GreenConc                      float64
lt                             float64
lg                             float64
priceLogScaled                 float64
totalScore                     float64
dtype: object

In [21]:
feature_group7 = folium.FeatureGroup(name='Objects (apartments)')


location = geolocator.geocode("Milano Italy")
# map2 = folium.Map(
#     location=[location.latitude, location.longitude],
#     tiles='cartodbpositron',
#     zoom_start=13)
# tooltip = 'Click me!'

for row in dfObjects.iterrows():
    link = "<a href='https://www.immobiliare.it/annunci/"+str(row[1].id.split("_")[2])+"/"+" target='_blank'" + ">Link</a>"
    st = f"{row[1].type},{row[1].id},EUR {row[1].price}"
    folium.Marker([row[1]["lt"], row[1]["lg"]], 
                  popup = st+" "+ link + " Total score:" + str(row[1].totalScore), 
                  tooltip=tooltip,
                  icon=folium.Icon(color=color_producer(row[1].totalScore, max(dfObjects["totalScore"]), 
                                                        inv = True))).add_to(feature_group7)


#feature_group6.add_to(mapGreen)
feature_groups.append(feature_group7)

### 8. Disadvantaged areas

In [22]:
def getLoc(address, annot = False):
    """
    
    """
    try:
        location = geolocator.geocode(address)
        if annot:
            print(address + " was succesfully added\n")
        return [location[1][0], location[1][1]]
        
    except:
        if annot:
            print(address + " was skipped. Probably was not found\n")
        return np.nan

In [23]:
feature_group8 = folium.FeatureGroup(name='Dangerous Zones')
blackList = ["Quarto Oggiaro", "Roserio", "viale Padova", "Bovisa", "Rogored", "Barona", "Corvetto", "San Siro", "Via Gola"]
for item in blackList:
    folium.Circle(getLoc(item),
                  radius = 2000,
                  fill = True,
                  fill_color = "yellow",
                  weight = 0,
                  popup = item, 
                  tooltip=tooltip).add_to(feature_group8)
    
    folium.Circle(getLoc(item),
                  radius = 1000,
                  fill = True,
                  fill_color = "red",
                  weight = 0,
                  popup = item, 
                  tooltip=tooltip).add_to(feature_group8)



#feature_group7.add_to(mapGreen)
feature_groups.append(feature_group8)

# folium.LayerControl(collapsed = False).add_to(mapGreen)
# mapGreen
# mapGreen.save("mapGreen.html") 

### 9. Heat map of predicted total score

In [24]:
dct = {"lt":[],
      "lg": []}
for i in np.arange(minmaxlat[0], minmaxlat[1], 0.01/0.75):
    for j in np.arange(minmaxlong[0], minmaxlong[1], 0.02):
        dct["lt"].append(i)
        dct["lg"].append(j)
        
dfHeatmap = pd.DataFrame(dct)
dfHeatmap = dfHeatmap.append(dfAir3[["lt","lg"]])
dfHeatmap = dfHeatmap.append(dfObjects[["lt","lg"]])
dfHeatmap

,lt,lg
0,45.390000,9.080000
1,45.390000,9.100000
2,45.390000,9.120000
3,45.390000,9.140000
4,45.390000,9.160000
...,...,...
1000,45.507237,9.162271
1012,45.481041,9.219500
1015,45.464544,9.142241
1028,45.512450,9.220291


In [25]:
import pickle
pkl_filename = "rfr2.pkl"
with open(pkl_filename, 'rb') as file:
    rfr2 = pickle.load(file)

In [26]:
dfHeatmap["totalScore"] = rfr2.predict(dfHeatmap)
dfHeatmap

,lt,lg,totalScore
0,45.390000,9.080000,4.154115
1,45.390000,9.100000,2.594608
2,45.390000,9.120000,2.071414
3,45.390000,9.140000,2.133632
4,45.390000,9.160000,2.213668
...,...,...,...
1000,45.507237,9.162271,4.730297
1012,45.481041,9.219500,2.176413
1015,45.464544,9.142241,4.107752
1028,45.512450,9.220291,0.893307


In [27]:
import branca.colormap as cm
from collections import defaultdict
steps = 50
#color_map=cm.linear.Blues_03.scale(0,1).to_step(steps)
color_map=cm.linear.RdBu_03.scale(0,1).to_step(steps)

feature_group9 = folium.FeatureGroup(name='Predicted Total Score')
gradient_map=defaultdict(dict)
for i in range(steps):
    gradient_map[1/steps*i] = color_map.rgb_hex_str(1/steps*i)

# mapB = folium.Map(
#     location=[location.latitude, location.longitude],
#     zoom_start=13,
#     tiles='cartodbpositron')

HeatMap(dfHeatmap, min_opacity = 0.5, 
        #radius =50,
        blur = 30,
       gradient = gradient_map).add_to(folium.FeatureGroup(name='Heat Map').add_to(feature_group9))

#feature_group6.add_to(mapGreen)
feature_groups.append(feature_group9)

### Plotting the map

In [28]:
for f in feature_groups:
    f.add_to(mapGreen)
folium.LayerControl(collapsed = False).add_to(mapGreen)
mapGreen
mapGreen.save("mapGreen.html") 

### 10. Forming an interactive Dash

In [29]:
import pandas as pd
import plotly.express as px  # (version 4.7.0)
import plotly.graph_objects as go

import dash  # (version 1.12.0) pip install dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output

In [30]:
app = dash.Dash(__name__)

In [31]:
app.layout = html.Div([
    html.H1("Where to live in Milan?"),
    dcc.Dropdown(id="slct_season",
                options = [{'label':'Bilocale', 'value':1},
                           {'label':'Trilocale', 'value':2},
                           {'label':'Loft', 'value':3},
                           {'label':'winter', 'value':4}],
                multi = False,
                value = 1),
    html.Div(id="output_container", children = [html.Iframe(srcDoc = open('mapGreen.html', 'r').read(),
                                                           style = {"border": 0, 
                                                                    "width":"100%", 
                                                                    "height": "800px",
                                                                    "overflow": "auto"})]),
    html.Br()#,
    #dcc.Graph(id='season_chart', figure = {})
])


In [32]:
@app.callback(
    [Output(component_id='output_container', component_property='children'),
     Output(component_id='my_bee_map', component_property='figure')],
    [Input(component_id='slct_year', component_property='value')]
)
def update_graph(option_slctd):
    print(option_slctd)
    print(type(option_slctd))

    container = "The year chosen by user was: {}".format(option_slctd)
    

    # Plotly Express
    fig = html.Iframe(srcDoc = open('map2.html', 'r').read())
    map2 = folium.Map(
    location=[location.latitude, location.longitude],
    tiles='cartodbpositron',
    zoom_start=13)
    tooltip = 'Click me!'

    for row in dfObjects.iterrows():
        link = "<a href='https://www.immobiliare.it/annunci/"+str(row[1].id.split("_")[2])+"/"+" target='_blank'" + ">Link</a>"
        st = f"{row[1].type},|,{row[1].id},EUR {row[1].price}"
        folium.Marker(row[1].lc, 
                      popup = st+" "+ link, 
                      tooltip=tooltip,
                      icon=folium.Icon(color=color_producer(row[1].totalScore, max(dfObjects["totalScore"]), inv = True))).add_to(feature_group6)


    feature_group6.add_to(map2)
    map2.save("map2.html")


    return container, fig

In [33]:
app.run_server()

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [21/Nov/2020 03:02:49] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [21/Nov/2020 03:02:49] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [21/Nov/2020 03:02:49] "GET /_dash-layout HTTP/1.1" 200 -
